# State Conservation Status and Sensitive Species Lists - VIC

This notebook downloads the VIC lists from the [Victorian Government Open Data Portal](https://vba.dse.vic.gov.au/vba/downloadVSC.do) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/VIC` directory, process the lists and save them to `current-lists`.
The lists can be loaded into the lists-test environment then reviewed by following the instructions below.

## Lists in the ALA Species List tool
* Conservation list: __[dr655](https://lists.ala.org.au/speciesListItem/list/dr655)__ (in [test](https://lists-test.ala.org.au/speciesListItem/list/dr655))
* Sensitive list: __[dr490](https://lists.ala.org.au/speciesListItem/list/dr490)__ (in [test](https://lists-test.ala.org.au/speciesListItem/list/dr18669))

* Collection: __[dr655](https://collections.ala.org.au/public/show/dr655)__ __[dr490](https://collections.ala.org.au/public/show/dr490)__

## Sources
Victorian Biodivesity Atlas - Victoria Department of Environment, Land, Water and Planning
See https://vba.biodiversity.vic.gov.au/vba

* __[Metadata (Open Data Portal)](https://discover.data.vic.gov.au/dataset/victorian-biodiversity-atlas-vba-taxa-list1)__
* __[VBA Metadata](https://vba.dse.vic.gov.au/vba/index.jsp)__ (requires sign in)
* __[Data](https://vba.dse.vic.gov.au/vba/downloadVSC.do)__

**Metadata Description**
This list is generated from the subset of species listed in the Victorian Biodiversity Atlas (VBA) Taxa List downloaded from https://vba.dse.vic.gov.au/vba/downloadVSC.do which have a conservation status attributed to them.

After the review of the Flora and Fauna Guarantee Act (FFG Act) all previously listed species under the three different legislations and policy have been re-assessed. The VBA has been updated to reflect the gazetted and published FFG Threatened List. The Conservation Status is now only EPBC and FFG. Vic Advisory Lists are now redundant but retained for your information as 'Old Advisory List'. This is the interim measure before the national Single Operational List is finalised, which will replace the 3 different lists previously held in Victoria. The FFG categories are the nationally agreed IUCN Red list terms plus the EPBC Conservation Dependent category.

The main attributes in this table are the current VBA taxon_id, scientific name, common name, authority, family (where relevant), conservation status, origin and taxon type. Where possible, taxon id's have been included that provide a match to the obsolete Atlas of Victorian Wildlife taxaid, and the Flora Infromation System.

**URL**
https://discover.data.vic.gov.au/dataset/victorian-biodiversity-atlas-vba-taxa-list1

Prerequisite: Data file has been downloaded into `source-data/VIC/Species-Checklist.csv`


In [3]:
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"

import pandas as pd
import sys
import os

# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

cdataDir = "current-lists/conservation-lists/"
sdataDir = "current-lists/sensitive-lists/"
vicchecklisturl = "https://vba.dse.vic.gov.au/vba/downloadVSC.do" # gets a csv
vicchecklist = pd.read_csv(projectDir + "source-data/VIC/Species-Checklist.csv",encoding='cp1252')


# Extract and format initial list
1. Drop non-required columns
2. Rename columns for Darwin Core list terms
3. Assign additional required list fields

In [4]:
vicchecklist['LAST_MOD'].drop_duplicates().sort_values()

22182    01/01/2010
10432    01/02/2012
11265    01/02/2018
12493    01/02/2021
18298    01/03/2017
            ...    
13112    31/10/2016
159      31/10/2017
10501    31/10/2018
11206    31/10/2020
15567    31/10/2022
Name: LAST_MOD, Length: 899, dtype: object

In [5]:
# Drop columns
vicchecklist = vicchecklist.drop(['PARENT_TAXON_LEVEL_CDE','PARENT_TAXON_ID','ALL_DISCIPLINE_CODES',
                                          'TREATIES','SHORT_NAME','NVIS_GROWTHFORM','SCIENTIFIC_NME_SYNONYM',
                                          'COMMON_NME_SYNONYM','PRINT_ORDER_NUM'],axis=1)
# Rename columns
vicchecklist = vicchecklist.rename(columns=
{
    'TAXON_ID':'taxonID',
    'SCIENTIFIC_NAME':'scientificName',
    'VIC_ADVISORY_STATUS':'vicadvisorystatus',
    'TAXON_TYPE':'speciesGroup',
    'FFG_ACT_STATUS':'ffgactstatus',
    'EPBC_ACT_STATUS': 'epbcactStatus',
    'TAXON_LEVEL_CDE': 'taxonRank',
    'ORIGIN': 'establishmentMeans',
    'COMMON_NME_SYNONYM': 'taxonRemarks',
    'COMMON_NAME':'vernacularName',
    'AUTHORITY':'scientificNameAuthority',
    'PRIMARY_DISCIPLINE': 'primaryDiscipline',
    'RESTRICTED_FLAG' : 'restrictedFlag',
    'EXTRACT_DATE': 'extractDate',
    'LAST_MOD':'modified'
})
vicchecklist['taxonRank'] = 'species'
vicchecklist['status'] = vicchecklist['ffgactstatus']
vicchecklist['sourceStatus'] = vicchecklist['ffgactstatus']

# reformat date
vicchecklist['extractDate'] = pd.to_datetime(vicchecklist['extractDate'], format='%d/%m/%Y')
vicchecklist['extractDate'] = vicchecklist['extractDate'].dt.strftime('%Y-%m-%d')
vicchecklist['modified'] = pd.to_datetime(vicchecklist['modified'], format='%d/%m/%Y')
vicchecklist['modified'] = vicchecklist['modified'].dt.strftime('%Y-%m-%d')

# Split out sensitive and conservation lists

In [6]:
sensitiveList = vicchecklist[vicchecklist["restrictedFlag"].notna()]
conservationList = vicchecklist[vicchecklist["status"].notna()]
c = [len(conservationList.index)]
d = {'sensitive list count': [len(sensitiveList.index)],
     'conservation list count':[len(conservationList.index)]}
pd.DataFrame(d).transpose()

,0
sensitive list count,136
conservation list count,1999


In [7]:
#Write to CSV
conservationList.to_csv(projectDir + cdataDir + "VIC-conservation.csv",  encoding="UTF-8", index=False)
sensitiveList.to_csv(projectDir + sdataDir + "VIC-sensitive.csv",  encoding="UTF-8", index=False)
print('Finished processing')

Finished processing


# Manual List check

**Instructions**
1. Load the lists above into the lists-test tool
2. Check the list name matching score and the text appearance on species pages
3. Unskip the below code and Run the reports below to compare to production. Send the changelog.csv to check. Correct any issues.
4. Save the production list into the `historical lists` directory by uncommenting the code section below.
5. Load the lists into production

### Conservation List - Download old and new and compare

In [8]:
# %%script echo skipping # comment this line to run this code

import datetime
monthStr = datetime.datetime.now().strftime('%Y%m')
ltype = "C"
# conservation
filename = "VIC-conservation.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr655" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr655" + "?max=10000&includeKVP=true"
changelist = lf.get_changelist(testListUrl, prodListUrl, ltype)
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

,name,commonName_new,scientificName_new,status_new,listUpdate,commonName_old,scientificName_old,status_old
36,Galaxias sp. 14,Galaxias,Galaxias terenasus,Critically Endangered,added,NaN,NaN,NaN
77,Hyridella (Hyridella) depressa,Freshwater Mussel,Hyridella (Hyridella) depressa,Endangered,added,NaN,NaN,NaN
79,Hyridella narracanensis,Freshwater Mussel,Hyridella (Hyridella) narracanensis,Endangered,added,NaN,NaN,NaN
985,Leichhardtia australis,Doubah,Marsdenia australis,Endangered,added,NaN,NaN,NaN
986,Leichhardtia flavescens,None,Leichhardtia flavescens,Critically Endangered,added,NaN,NaN,NaN
1148,Orianthera pusilla,None,Orianthera pusilla,Vulnerable,added,NaN,NaN,NaN
1317,Prasophyllum sp. aff. montanum,None,Prasophyllum sp. montane (D.T.Rouse 180),Endangered,added,NaN,NaN,NaN
1320,Prasophyllum sp. aff. validum,None,Prasophyllum sp. Woodlands (D.L.Jones 17707),Endangered,added,NaN,NaN,NaN
1039,Logania pusilla,NaN,NaN,NaN,removed,None,Orianthera pusilla,Vulnerable
1060,Marsdenia australis,NaN,NaN,NaN,removed,Doubah,Marsdenia australis,Endangered


 ### Download Production lists to Historical Lists directory

In [9]:
prodList = lf.download_ala_list(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
print('Finished downloading historical list')

Finished downloading historical list


### Sensitive List - Download old and new and compare

In [10]:
# %%script echo skipping # comment this line to run this code
ltype = "S"
filename = "VIC-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr490" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr18669" + "?max=10000&includeKVP=true"
changelist = lf.get_changelist(testListUrl, prodListUrl, ltype )
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist
## check this as no list in test at the moment

,name,commonName_new,scientificName_new,listUpdate,commonName_old,scientificName_old
52,Caladenia arenaria,Sand-hill Spider-orchid,Caladenia arenaria,added,NaN,NaN
84,Prasophyllum retroflexum,Kiandra Leek Orchid,Prasophyllum retroflexum,added,NaN,NaN
99,Callocephalon fimbriatum,Gang-gang Cockatoo,Callocephalon fimbriatum,added,NaN,NaN
114,Miniopterus orianae bassanii,Southern Bent-winged Bat,Miniopterus orianae bassanii,added,NaN,NaN
60,Prasophyllum sp. aff. validum A,NaN,NaN,removed,None,Prasophyllum sp. Woodlands (D.L.Jones 17707)


### Download Production lists to Historical Lists directory

In [11]:
prodList = lf.download_ala_list(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)
print('Finished downloading historical list')

Finished downloading historical list
